### Beautiful Soup
Escribe una función llamada scrape_coches_ocasion que acepta una URL como argumento y devuelve un DataFrame de pandas con información sobre coches de ocasión encontrados en la página web especificada. La función realizará las siguientes tareas:
- Realizará una solicitud HTTP GET a la 
- Comprobará el código de estado de la respuesta. Si el código no es 200 (OK), imprimirá un mensaje de error y retornará None.
- Extraerá los nombres de los coches, los precios rebajados, los precios originales y la información de los kilómetros recorridos de la página
- Organizará los datos extraídos en un DataFrame de pandas con las siguientes columnas:
        "coche": Nombres de los coches.
        "precio_rebajado": Precios rebajados
        "precio_original": Precios originales
        "km": Kilómetros recorridos
- Retornará el DataFrame resultante
- Una vez extraida toda la información deberéis almacenar toda la información en un DataFrame



In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
url_coches = "https://www.ocasionplus.com/coches-ocasion?type%3DCAR%26sort%3DFECHAOP_DESC%26page%3D2"
coches = requests.get(url_coches)
print("La respuesta de la petición es:", coches.status_code)

La respuesta de la petición es: 200


In [5]:
coches_soup = BeautifulSoup(coches.content, 'html.parser')
#coches_soup
#print (coches_soup.prettify())

In [6]:
lista_nombre_coche = coches_soup.find_all ("h2", {"style": "overflow-x:hidden"})
lista_nombre_coche[:4]

[<h2 style="overflow-x:hidden">SEAT Leon ST 1.2 TSI S&amp;S Style (110 CV)</h2>,
 <h2 style="overflow-x:hidden">Peugeot 2008 SUV BlueHDi 100 Allure (100 CV)</h2>,
 <h2 style="overflow-x:hidden">Toyota C-HR 2.0 180H Advance (184 CV)</h2>,
 <h2 style="overflow-x:hidden">Fiat 500C Sport Cabrio (69 CV)</h2>]

In [7]:
lista_nombre_coche[0].text

'SEAT Leon ST 1.2 TSI S&S Style (110 CV)'

In [8]:
nombre_coches = []

for nombre in lista_nombre_coche:
    nombre_coches.append (nombre.text)
nombre_coches [:4]

['SEAT Leon ST 1.2 TSI S&S Style (110 CV)',
 'Peugeot 2008 SUV BlueHDi 100 Allure (100 CV)',
 'Toyota C-HR 2.0 180H Advance (184 CV)',
 'Fiat 500C Sport Cabrio (69 CV)']

In [9]:
nombre_coches2 = [nombre.text for nombre in lista_nombre_coche]
nombre_coches2 [:4]

['SEAT Leon ST 1.2 TSI S&S Style (110 CV)',
 'Peugeot 2008 SUV BlueHDi 100 Allure (100 CV)',
 'Toyota C-HR 2.0 180H Advance (184 CV)',
 'Fiat 500C Sport Cabrio (69 CV)']

## Precio descuento

In [10]:
precio_descuento = coches_soup.findAll ("span", {"class": "jsx-2689423372 price"})
precio_descuento[:4]

[<span class="jsx-2689423372 price">10.900<!-- -->€</span>,
 <span class="jsx-2689423372 price">8.000<!-- -->€</span>,
 <span class="jsx-2689423372 price">22.636<!-- -->€</span>,
 <span class="jsx-2689423372 price">8.954<!-- -->€</span>]

In [11]:
#precio descuento
lista_precio_descuento = []

for precio in precio_descuento:
    precio = precio.text
    precio_limpio = int(precio.split()[0].replace(".", "").replace("€", ""))
    lista_precio_descuento.append(precio_limpio)
    
lista_precio_descuento [:4]

[10900, 8000, 22636, 8954]

## Precio original

In [12]:
precio_original = coches_soup.find_all ("span", {"class": "jsx-2870186023 priceLast"})
precio_original[:4]

[<span class="jsx-2870186023 priceLast">11.990<!-- -->€</span>,
 <span class="jsx-2870186023 priceLast">8.800<!-- -->€</span>,
 <span class="jsx-2870186023 priceLast">24.900<!-- -->€</span>,
 <span class="jsx-2870186023 priceLast">9.850<!-- -->€</span>]

In [13]:
#precio descuento
lista_precio_original = []

for precio in precio_original:
    precio = precio.text
    precio_limpio = int(precio.split()[0].replace(".", "").replace("€", ""))
    lista_precio_original.append(precio_limpio)
    
lista_precio_original [:4]

[11990, 8800, 24900, 9850]

## Info kms

In [14]:
kms = coches_soup.find_all ("spa", {"class": "jsx-3960002075 icon"}) #no funcciona
kms [0:4]

[]

In [29]:
import re

In [35]:
kms = coches_soup.find_all ("span", {"class": "jsx-3960002075 detail big"})

kms_lista = []

for km in kms:
    km = km.text
    km_limpio = re.findall("\d{1,3}\.\d{3}",km)
    
    if len(km_limpio) > 0:
        kms_lista.append (km)

kms_lista[0:30]

['145.772 km',
 '169.229 km',
 '31.442 km',
 '88.922 km',
 '31.842 km',
 '17.108 km',
 '123.362 km',
 '55.786 km',
 '47.382 km',
 '63.204 km',
 '81.070 km',
 '144.376 km',
 '18.502 km',
 '61.185 km',
 '44.136 km',
 '53.662 km',
 '65.962 km',
 '56.343 km',
 '38.851 km',
 '93.079 km']

In [38]:
kms_lista2 = []

for km in kms_lista:
    km = km.replace (" km",""). replace (".",",")
    kms_lista2.append(km)
kms_lista2


['145,772',
 '169,229',
 '31,442',
 '88,922',
 '31,842',
 '17,108',
 '123,362',
 '55,786',
 '47,382',
 '63,204',
 '81,070',
 '144,376',
 '18,502',
 '61,185',
 '44,136',
 '53,662',
 '65,962',
 '56,343',
 '38,851',
 '93,079']

In [40]:
#Todo de una vez

kms = coches_soup.find_all ("span", {"class": "jsx-3960002075 detail big"})

kms_lista = []

for km in kms:
    km = km.text
    km_limpio = re.findall("\d{1,3}\.\d{3}",km)
    
    if len(km_limpio) > 0:
        kms_lista.append (km)

        kms_lista3 = []

        for km in kms_lista:
            km = km.replace (" km",""). replace (".",",")
            kms_lista3.append(km)
        kms_lista3

In [41]:
kms_lista3

['145,772',
 '169,229',
 '31,442',
 '88,922',
 '31,842',
 '17,108',
 '123,362',
 '55,786',
 '47,382',
 '63,204',
 '81,070',
 '144,376',
 '18,502',
 '61,185',
 '44,136',
 '53,662',
 '65,962',
 '56,343',
 '38,851',
 '93,079']

In [24]:
patron = "\d{1,3}\.\d{3}"
lista_kms = re.findall(patron, kms)

TypeError: expected string or bytes-like object

In [20]:
lista_kms

('\\d{1,3}\\.\\d{3}', [])

In [17]:
kms = coches_soup.find_all ("span", {"img": "alt=Km"}) #no funcciona
c

NameError: name 'c' is not defined

In [ ]:
kms = coches_soup.find_all ("span", {"class": "jsx-3960002075 icon"}) #no funccionav
kms [0:20]

[]

In [ ]:
def scrape_coches (url):
    #1. Llamar a al sitio web y el código
    url = "https://www.ocasionplus.com/coches-ocasion?type%3DCAR%26sort%3DFECHAOP_DESC%26page%3D2"
    coches = requests.get(url)
    print("La respuesta de la petición es:", coches.status_code)

    #2. Definir el Beautiful Soup
    coches_beautiful = BeautifulSoup(coches.content, 'html.parser')
    